In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/awajun-spanish/train.es
/kaggle/input/awajun-spanish/train.agr
/kaggle/input/awajun-spanish/test.agr-es.src
/kaggle/input/awajun-spanish/dev.es
/kaggle/input/awajun-spanish/test.es-agr.src
/kaggle/input/awajun-spanish/dev.agr
/kaggle/input/aymara-spanish/train.es
/kaggle/input/aymara-spanish/test.aym-es.src
/kaggle/input/aymara-spanish/test.es-aym.src
/kaggle/input/aymara-spanish/dev.aym
/kaggle/input/aymara-spanish/dev.es
/kaggle/input/aymara-spanish/train.aym
/kaggle/input/otomi-spanish/test.es-oto.src
/kaggle/input/otomi-spanish/train.es
/kaggle/input/otomi-spanish/test.oto-es.src
/kaggle/input/otomi-spanish/train.oto
/kaggle/input/otomi-spanish/dev.oto
/kaggle/input/otomi-spanish/dev.es


In [2]:
%%capture
%pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu -q

Import libraries

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, NllbTokenizer
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
from sklearn.model_selection import train_test_split
import re
import random
from tqdm.auto import tqdm
import os

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `basic task` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `basic task`


## Dataset

In [5]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/awajun-spanish/train.agr', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/awajun-spanish/train.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
train_df  = pd.DataFrame({
    # 'Source': source,
    'Awajun': ashaninka_lines,
    'Spanish': spanish_lines
})

In [6]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/awajun-spanish/dev.agr', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/awajun-spanish/dev.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
dev_df  = pd.DataFrame({
    # 'Source': source,
    'Awajun': ashaninka_lines,
    'Spanish': spanish_lines
})

In [7]:
# Create a test set from the dev data
dev_df, test_df = train_test_split(dev_df, test_size=0.5, random_state=42)

In [8]:
print(f"Training samples: {len(train_df)}")
print(f"Development samples: {len(dev_df)}")
print(f"Test samples: {len(test_df)}")

Training samples: 21964
Development samples: 509
Test samples: 509


In [9]:
# Step 4: Initialize the NLLB tokenizer and model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = NllbTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Examine tokenization

In [10]:
def word_tokenize(text):
    return re.findall('(\w+|[^\w\s])', text)

In [11]:
# Sample from the training data
sample = train_df.sample(min(10000, len(train_df)), random_state=1)

In [12]:
# Apply tokenization
sample['agr_toks'] = sample.Awajun.apply(tokenizer.tokenize)
sample['spa_toks'] = sample.Spanish.apply(tokenizer.tokenize)
sample['agr_words'] = sample.Awajun.apply(word_tokenize)
sample['spa_words'] = sample.Spanish.apply(word_tokenize)

In [13]:
# Compute statistics
stats = sample[['agr_toks', 'spa_toks', 'agr_words', 'spa_words']].applymap(len).describe()

<ipython-input-13-aade89a857c6>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stats = sample[['agr_toks', 'spa_toks', 'agr_words', 'spa_words']].applymap(len).describe()


In [14]:
print("Average tokens per word for Awajun:", stats.agr_toks['mean'] / stats.agr_words['mean'])
print("Average tokens per word for Spanish:", stats.spa_toks['mean'] / stats.spa_words['mean'])

Average tokens per word for Awajun: 2.204560313053695
Average tokens per word for Spanish: 1.1994777535241437


In [15]:
# Check for unknown tokens
texts_with_unk_agr = [
    text for text in tqdm(train_df.Awajun)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

texts_with_unk_spa = [
    text for text in tqdm(train_df.Spanish)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

  0%|          | 0/21964 [00:00<?, ?it/s]

  0%|          | 0/21964 [00:00<?, ?it/s]

In [16]:
print(f"Number of Awajun texts with unknown tokens: {len(texts_with_unk_agr)}")
print(f"Number of Spanish texts with unknown tokens: {len(texts_with_unk_spa)}")

Number of Awajun texts with unknown tokens: 6131
Number of Spanish texts with unknown tokens: 133


## Add Language Token if needed

In [17]:
def fix_tokenizer(tokenizer, new_lang='agr_Latn'):
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset
    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [18]:
if 'agr_Latn' not in tokenizer.lang_code_to_id:
    fix_tokenizer(tokenizer, 'agr_Latn')
    model.resize_token_embeddings(len(tokenizer))
    
    # Initialize the embedding for the new token with a similar language
    added_token_id = tokenizer.convert_tokens_to_ids('agr_Latn')
    similar_lang_id = tokenizer.convert_tokens_to_ids('quy_Latn')  # Quechua is similar to Aymara
    embeds = model.model.shared.weight.data
    embeds[added_token_id] = embeds[similar_lang_id]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


## Preprocess function for text

In [19]:
def preproc(text):
    from sacremoses import MosesPunctNormalizer
    mpn = MosesPunctNormalizer(lang="en")
    clean = mpn.normalize(text)
    return clean

## Training function

In [20]:
def get_batch_pairs(batch_size, data=train_df):
    LANGS = [('Awajun', 'agr_Latn'), ('Spanish', 'spa_Latn')]
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

In [21]:
def train_model(model, tokenizer, training_steps=20000, batch_size=16, max_length=128, lr=1e-4):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Define optimizer and scheduler
    optimizer = Adafactor(
        [p for p in model.parameters() if p.requires_grad],
        scale_parameter=False,
        relative_step=False,
        lr=lr,
        clip_threshold=1.0,
        weight_decay=1e-3,
    )
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)
    
    # Training loop
    losses = []
    MODEL_SAVE_PATH = 'nllb-aym-spa-v1'
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    model.train()
    x, y, loss = None, None, None
    
    def cleanup():
        import gc
        gc.collect()
        torch.cuda.empty_cache()
    
    for i in tqdm(range(training_steps)):
        xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
        
        try:
            tokenizer.src_lang = lang1
            x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            tokenizer.src_lang = lang2
            y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100
            
            loss = model(**x, labels=y.input_ids).loss
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            
        except RuntimeError as e:
            optimizer.zero_grad(set_to_none=True)
            x, y, loss = None, None, None
            cleanup()
            print('error', max(len(s) for s in xx + yy), e)
            continue
        
        if i % 1000 == 0:
            avg_loss = torch.mean(torch.tensor(losses[-1000:] if len(losses) > 1000 else losses))
            print(f"Step {i}, Average Loss: {avg_loss}")
            if i % 5000 == 0 and i > 0:
                model.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
                tokenizer.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
    
    # Save the final model
    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)
    
    return model, tokenizer


## Evaluation function

In [22]:
def evaluate_model(model, tokenizer, test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Function to translate text
    def translate(text, src_lang, tgt_lang, num_beams=4):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate Aymara to Spanish
    agr_to_spa = []
    for text in tqdm(test_data.Awajun):
        agr_to_spa.append(translate(text, 'agr_Latn', 'spa_Latn')[0])
    
    # Translate Spanish to Aymara
    spa_to_agr = []
    for text in tqdm(test_data.Spanish):
        spa_to_agr.append(translate(text, 'spa_Latn', 'agr_Latn')[0])
    
    # Calculate BLEU scores
    import sacrebleu
    
    bleu_agr_spa = sacrebleu.corpus_bleu(agr_to_spa, [test_data.Spanish.tolist()])
    bleu_spa_agr = sacrebleu.corpus_bleu(spa_to_agr, [test_data.Awajun.tolist()])
    
    chrf_agr_spa = sacrebleu.corpus_chrf(agr_to_spa, [test_data.Spanish.tolist()])
    chrf_spa_agr = sacrebleu.corpus_chrf(spa_to_agr, [test_data.Awajun.tolist()])
    
    # Calculate exact match accuracy
    agr_spa_exact_matches = sum(1 for pred, ref in zip(agr_to_spa, test_data.Spanish) if pred.strip() == ref.strip())
    spa_agr_exact_matches = sum(1 for pred, ref in zip(spa_to_agr, test_data.Awajun) if pred.strip() == ref.strip())
    
    agr_spa_accuracy = (agr_spa_exact_matches / len(test_data)) * 100
    spa_agr_accuracy = (spa_agr_exact_matches / len(test_data)) * 100
    
    # Calculate word-level accuracy (percentage of words that match)
    def calculate_word_accuracy(predictions, references):
        total_words = 0
        correct_words = 0
        
        for pred, ref in zip(predictions, references):
            pred_words = pred.strip().split()
            ref_words = ref.strip().split()
            
            # Count words in reference
            total_words += len(ref_words)
            
            # Count matching words (position-sensitive)
            for i, ref_word in enumerate(ref_words):
                if i < len(pred_words) and pred_words[i] == ref_word:
                    correct_words += 1
        
        return (correct_words / total_words) * 100 if total_words > 0 else 0
    
    agr_spa_word_accuracy = calculate_word_accuracy(agr_to_spa, test_data.Spanish)
    spa_agr_word_accuracy = calculate_word_accuracy(spa_to_agr, test_data.Awajun)
    
    print(f"BLEU Awajun → Spanish: {bleu_agr_spa.score:.2f}")
    print(f"BLEU Spanish → Awajun: {bleu_spa_agr.score:.2f}")
    print(f"chrF++ Awajun → Spanish: {chrf_agr_spa.score:.2f}")
    print(f"chrF++ Spanish → Awajun: {chrf_spa_agr.score:.2f}")
    print(f"Exact Match Accuracy Awajun → Spanish: {agr_spa_accuracy:.2f}%")
    print(f"Exact Match Accuracy Spanish → Awajun: {spa_agr_accuracy:.2f}%")
    print(f"Word-level Accuracy Awajun → Spanish: {agr_spa_word_accuracy:.2f}%")
    print(f"Word-level Accuracy Spanish → Awajun: {spa_agr_word_accuracy:.2f}%")
    
    return {
        'bleu_agr_spa': bleu_agr_spa.score,
        'bleu_spa_agr': bleu_spa_agr.score,
        'chrf_agr_spa': chrf_agr_spa.score,
        'chrf_spa_agr': chrf_spa_agr.score,
        'agr_spa_accuracy': agr_spa_accuracy,
        'spa_agr_accuracy': spa_agr_accuracy,
        'agr_spa_word_accuracy': agr_spa_word_accuracy,
        'spa_agr_word_accuracy': spa_agr_word_accuracy,
        'agr_to_spa_samples': list(zip(test_data.Awajun.head(5), agr_to_spa[:5])),
        'spa_to_agr_samples': list(zip(test_data.Spanish.head(5), spa_to_agr[:5]))
    }

## Main Execution

In [23]:
# Train the model
print("Starting model training...")
model, tokenizer = train_model(model, tokenizer, training_steps=30000) #Min 10000 steps need to be a good translator

Starting model training...


  0%|          | 0/30000 [00:00<?, ?it/s]

Step 0, Average Loss: 8.250648498535156
Step 1000, Average Loss: 3.8847758769989014
Step 2000, Average Loss: 2.857036828994751
Step 3000, Average Loss: 2.481383800506592
Step 4000, Average Loss: 2.2352676391601562
Step 5000, Average Loss: 2.0699994564056396
Step 6000, Average Loss: 1.940630316734314
Step 7000, Average Loss: 1.8420625925064087
Step 8000, Average Loss: 1.7350778579711914
Step 9000, Average Loss: 1.6464824676513672
Step 10000, Average Loss: 1.5699949264526367
Step 11000, Average Loss: 1.5015307664871216
Step 12000, Average Loss: 1.4339877367019653
Step 13000, Average Loss: 1.362600564956665
Step 14000, Average Loss: 1.309276819229126
Step 15000, Average Loss: 1.2480547428131104
Step 16000, Average Loss: 1.2103123664855957
Step 17000, Average Loss: 1.160968542098999
Step 18000, Average Loss: 1.1081782579421997
Step 19000, Average Loss: 1.0401508808135986
Step 20000, Average Loss: 1.006708025932312
Step 21000, Average Loss: 0.9682196974754333
Step 22000, Average Loss: 0.938

In [24]:
# Evaluate the model
print("Evaluating the model...")
eval_results = evaluate_model(model, tokenizer, test_df)

Evaluating the model...


  0%|          | 0/509 [00:00<?, ?it/s]

  0%|          | 0/509 [00:00<?, ?it/s]

BLEU Awajun → Spanish: 11.12
BLEU Spanish → Awajun: 5.97
chrF++ Awajun → Spanish: 32.80
chrF++ Spanish → Awajun: 38.59
Exact Match Accuracy Awajun → Spanish: 2.36%
Exact Match Accuracy Spanish → Awajun: 1.18%
Word-level Accuracy Awajun → Spanish: 6.95%
Word-level Accuracy Spanish → Awajun: 5.59%


## Sample Testing

In [25]:
# Print some translation examples
print("\Awajun to Spanish translation examples:")
for src, tgt in eval_results['agr_to_spa_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

print("\nSpanish to Awajun translation examples:")
for src, tgt in eval_results['spa_to_agr_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

\Awajun to Spanish translation examples:
Source: Yatsug aidauh, ubag aidauh, iina muunji aajakajua nunú, ashí yujagminum ijiam nayants kapantu tutainum katiajua nunú kajimatkijum tusan dakitajai.
Translation: Porque es mi deseo, hermanos, que tengan en mente la memoria, cómo nuestros padres pasaron por el medio del Mar Rojo en una nube.

Source: Sása wajasé wampá néje.
Translation: Se ha mojado la carne.

Source: Tuja Jabígka suntaji aina nuna apuji Sísara, atumin jetepaktatus niina carroji aidaunum namak Quisónnum kaunatnume titatjai. Nunikmatai wi atumin idaitustatjime depetuk ijinatnume’, tusa Tuke Pujuu tujamui”, tiuwai.
Translation: Y haré que Sísara, el capitán de los carros de guerra de Jabín, venga contra ustedes al río Cisón, y yo los entregaré en sus manos, y los entregaré en sus manos, dice el Señor.

Source: Tuke batsamin ikamia: Ashi ikam batsamin aidau tuke batsamin aina nuwi pachitkau aidau, ijunag nuniachkush tupagka batsamin.
Translation: Pueblos indígenas: Pueblos ori

## Hugging Face push model

In [26]:
model_name = "mahsharyahan/nllb-600B-agr-spa-v1"
tokenizer.push_to_hub(model_name)
model.push_to_hub(model_name)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahsharyahan/nllb-600B-agr-spa-v1/commit/866e1274f87d1562e7c044698e613f771d64ee5a', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='866e1274f87d1562e7c044698e613f771d64ee5a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mahsharyahan/nllb-600B-agr-spa-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='mahsharyahan/nllb-600B-agr-spa-v1'), pr_revision=None, pr_num=None)

## Submit Function

In [27]:
def test_translation_without_targets(model, tokenizer, test_data, src_lang, tgt_lang, num_samples=5):
    import torch
    from tqdm.auto import tqdm
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Determine column name based on language direction
    if src_lang == 'agr_Latn':
        src_col = 'Awajun'
    else:
        src_col = 'Spanish'
    
    # Function to translate text
    def translate(text, num_beams=4, max_length=128):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams,
                max_length=max_length
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate test samples
    translations = []
    
    # Select random samples for detailed review
    test_samples = test_data.sample(min(num_samples, len(test_data)))
    
    # Process all test data
    print(f"\n===== Translation Results: {src_lang} to {tgt_lang} =====")
    print("Sample Translations:")
    
    for i, (_, row) in enumerate(tqdm(test_data.iterrows(), total=len(test_data), desc=f"Translating {src_lang} to {tgt_lang}")):
        source_text = row[src_col]
        translated_text = translate(source_text)[0]
        translations.append(translated_text)
        
        # Display detailed examples for the selected samples
        if _ in test_samples.index:
            print(f"\nExample {i+1}:")
            print(f"Source ({src_col}): {source_text}")
            print(f"Translation: {translated_text}")
    
    return translations

In [28]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/awajun-spanish/test.agr-es.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_agr_es = pd.DataFrame({
    # 'Source': source,
    'Awajun': ashaninka_lines,
})

In [29]:
agr_to_es_results = test_translation_without_targets(model, tokenizer, df_test_agr_es, 'agr_Latn', 'spa_Latn')


===== Translation Results: agr_Latn to spa_Latn =====
Sample Translations:


Translating agr_Latn to spa_Latn:   0%|          | 0/358 [00:00<?, ?it/s]


Example 175:
Source (Awajun): Ninahuanca chichaak interdicción juju nugka nuwig atsawai tawai
Translation: La Ninjauca señala que ninguna interdicción existe en el territorio nacional

Example 244:
Source (Awajun): Juju mediowa nunu Hortez Baitugkan untsuke- niinak Aogustina Mayan nuwigtu Zebelio Kayap  minero ilegal aidaun utsaa ditajai takawai tuinawai- niina chichamen dekajuatatus
Translation: El medio ha convocado a Hortez Baituk - quien agrupa a Aogustin Mayan y Zebelio Caif- para investigar la situación

Example 335:
Source (Awajun): Nuwauchik junio nagkaemakma nuwi jakabi, nueve tsawan hospital Santa Rosa Puerto Maldonadonum awayam tepesa
Translation: La paciente falleció en el último mes de junio, luego de ser atendida en el Hospital Santa Rosa de Port Maldonado, a unos 9 días de distancia.

Example 345:
Source (Awajun): Kuntinun maatatus weenakug ikam kuwashat tsawan weenawai
Translation: Viaja los bosques por varios días para matar

Example 347:
Source (Awajun): najankamu ai

In [30]:
# Create a DataFrame
df_test_result_agr_to_es = pd.DataFrame({
    # 'Source': source,
    'Spanish': agr_to_es_results,
})

In [31]:
df_test_result_agr_to_es.to_csv('test_result_agr_to_es.tsv', sep='\t', index=False, header=False)

In [32]:
from IPython.display import FileLink

FileLink("test_result_agr_to_es.tsv")

/kaggle/working/test_result_agr_to_es.tsv

In [33]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/awajun-spanish/test.es-agr.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_es_agr = pd.DataFrame({
    # 'Source': source,
    'Spanish': ashaninka_lines,
})

In [34]:
es_to_agr_results = test_translation_without_targets(model, tokenizer, df_test_es_agr, 'spa_Latn','agr_Latn')


===== Translation Results: spa_Latn to agr_Latn =====
Sample Translations:


Translating spa_Latn to agr_Latn:   0%|          | 0/358 [00:00<?, ?it/s]


Example 75:
Source (Spanish): En marzo, abril y mayo de 2020 estábamos en el primer pico de la enfermedad, y la mayor proporción de los internados era personas mayores de 60 años
Translation: Nantu marzo, abril nuwigtu mai mijan 2020tin jatak ima senchi waituidau asaja batsamiaji, nuniamunum ima kuwashat aents 60 mijan inagkeaki wayaa batsamiaja nuwi

Example 127:
Source (Spanish): Dibuja un triángulo.
Translation: Ukuta kampatum dakumka najanata.

Example 149:
Source (Spanish): ¿Cuál es su nombre?
Translation: ¿Amesh yaita?

Example 179:
Source (Spanish): Encuentra los pronombres de tercera persona en el texto.
Translation: Papii agagbaunum tikich aents adaikamu aidau adutuata.

Example 291:
Source (Spanish): Cuando amenazaron a Arbildo pedimos protección, pero nadie nos hizo caso, nos ignoraron
Translation: Arbidon ishamainakug ayamjukmi tusa segabiaji, tujash makichkish antugkachaju


In [35]:
# Create a DataFrame
df_test_result_es_to_agr = pd.DataFrame({
    # 'Source': source,
    'Awajun': es_to_agr_results,
})

In [36]:
df_test_result_es_to_agr.to_csv('test_result_es_to_agr.tsv', sep='\t', index=False, header=False)

In [37]:
from IPython.display import FileLink

FileLink("test_result_es_to_agr.tsv")

/kaggle/working/test_result_es_to_agr.tsv